### ДЗ 1
Имеются данные adult.csv (см. в материалах к занятию).  
Целевой переменной является уровень дохода income (крайний правый столбец).  
Описание признаков можно найти по ссылке www.cs.toronto.edu...etail.html  
Вам необходимо построить модели логистической регрессии и SVM, которые предсказывает уровень дохода человека.  
Вывести качество полученных моделей на тестовой выборке, используя функцию score у модели.  
Готовый ноутбук выложить на гитхаб и прислать ссылку.  

In [29]:
import numpy as np
import pandas as pd # Для работы с данными
import scipy.stats # При работе со статистикой
import matplotlib.pyplot as plt  # Библиотека для визуализации результатов 

data = pd.read_csv(r'E:\dvp-15\01 - Regression\Data\adult.csv') # датасет 

In [42]:
data.head(7)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0,0,30,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0,0,40,United-States,<=50K


In [4]:
#Видно что в данных есть пропуски, но при этом по Инфо все заполнено.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   age              48842 non-null  int64 
 1   workclass        48842 non-null  object
 2   fnlwgt           48842 non-null  int64 
 3   education        48842 non-null  object
 4   educational-num  48842 non-null  int64 
 5   marital-status   48842 non-null  object
 6   occupation       48842 non-null  object
 7   relationship     48842 non-null  object
 8   race             48842 non-null  object
 9   gender           48842 non-null  object
 10  capital-gain     48842 non-null  int64 
 11  capital-loss     48842 non-null  int64 
 12  hours-per-week   48842 non-null  int64 
 13  native-country   48842 non-null  object
 14  income           48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [90]:
#Подготовка данных в датасете
#data=data.replace('?',0)
data.replace(to_replace = r'?', value = np.nan, inplace=True)
#Объем датасета достаточно больше... дропаем все NaN
data.dropna(inplace=True)

In [91]:
data.tail(7)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
48835,53,Private,321865,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,>50K
48836,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K
48841,52,Self-emp-inc,287927,HS-grad,9,Married-civ-spouse,Exec-managerial,Wife,White,Female,15024,0,40,United-States,>50K


Для построения модели возмем основные значимые поля:
1. Класс работы (К)
2. Образование(цифровой вариант) (Num)
3. Получение капитала (Num)
4. Потеря капитала (Num)
5. Часы работы (Num)
6. Раса (К)
7. Работа (К)
8. Страна рождения (К)
9. Семейное положение (К)

In [68]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
le.fit(data.income)
y = pd.Series (data = le.transform(data.income))

In [106]:
le_X1 = LabelEncoder()
le_X2 = LabelEncoder()
le_X3 = LabelEncoder()
le_X4 = LabelEncoder()
le_X5 = LabelEncoder()

In [109]:
le_X1.fit(data['workclass'])
le_X2.fit(data['marital-status'])
le_X3.fit(data['relationship'])
le_X4.fit(data['gender'])
le_X5.fit(data['native-country'])

LabelEncoder()

In [111]:
selectedColumns = data [['workclass', 'marital-status','relationship','gender','native-country','educational-num','capital-gain','capital-loss','hours-per-week']]
X = pd.get_dummies (selectedColumns, columns = ['workclass', 'marital-status','relationship','gender','native-country'])

In [113]:
from sklearn.pipeline import make_pipeline # используем пайплайны для удобства
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=1000))

In [119]:
model2 = make_pipeline(LogisticRegression(max_iter=1000))

In [114]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [120]:
# обучаем на части датасета (train)

model.fit(X_train, y_train)
predictions = model.predict(X_test)

model2.fit(X_train, y_train)
predictions = model2.predict(X_test)

In [122]:
print(model.score(X_train, y_train),model2.score(X_train, y_train))

0.8385089243405465 0.8391723266466593


In [123]:
print (model.score(X_test,y_test),model2.score(X_test,y_test))

0.8433699417704724 0.8444755657109162


### SVM метод опорных векторов

In [124]:
from sklearn.svm import SVC
# используем пайплайны
clf = make_pipeline(StandardScaler(), SVC(gamma='auto')) 
clf.fit(X_train, y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', SVC(gamma='auto'))])

In [125]:
clf.score(X_train, y_train)

0.8437845522034434

In [126]:
clf.score(X_test, y_test)

0.8446229822363087